# Exploring GPyTorch & KeOps for determinants

In [1]:
!pip install -q torch gpytorch pykeops

In [2]:
import torch
from gpytorch import lazy
torch.set_default_dtype(torch.double)

## Naive dense implementation

Let us begin by computing the partition function of a phylogeny-constrained spanning tree distribution.

In [3]:
n = 3000  # number of leaves
d = 10  # embedding dimension

V = torch.arange(n)
U = torch.arange(n, 2 * n - 1)

torch.manual_seed(0)
t_V = torch.randn(n)
t_U = torch.randn(n - 1) - 1
t = torch.cat([t_V, t_U])

z = torch.randn(2 * n - 1, d)

In [4]:
dt = t[:, None] - t[None, :]
abs_dt = dt.abs().clamp(min=1e-2)  # avoid nans
dz = z[:, None] - z[None, :]
w = dz.pow(2).sum(-1).div(abs_dt).mul(-0.5).exp() / abs_dt.pow(d / 2)

# Exclude leaf-leaf edges.
is_leaf = torch.cat([torch.ones(n), torch.zeros(n - 1)])
w = w * (1 - is_leaf * is_leaf[:, None])

# Exclude internal-leaf edges that are out of order.
ooo = is_leaf[:, None] * (1 - is_leaf) * (dt <= 0).float()
w = w * (1 - ooo - ooo.transpose(-1, -2))

L = w.sum(dim=-1).diag_embed() - w

In [5]:
%%time
L[:-1, :-1].logdet()

CPU times: user 2.88 s, sys: 132 ms, total: 3.01 s
Wall time: 2.55 s


tensor(10864.2109)

In [6]:
%%time
L[:-1, :-1].cholesky().diag().log().sum()

CPU times: user 1.88 s, sys: 159 ms, total: 2.04 s
Wall time: 1.51 s


tensor(5432.1054)

## Speed up using GPyTorch's approximation

In [7]:
L_ = lazy.NonLazyTensor(L)

In [8]:
%%time
L_[:-1, :-1].logdet()

CPU times: user 1.28 s, sys: 10.7 ms, total: 1.3 s
Wall time: 1.28 s


tensor(11485.1592)

## Reduce memory using KeOps

In [9]:
from pykeops.torch import LazyTensor

if False:  # TODO
    t_ = LazyTensor(t)
    z_ = LazyTensor(z)
    
    dt = t_ - t_.transpose(-1, -2)
    abs_dt = dt.abs().clamp(min=1e-4)